In [32]:
import os
from dotenv import load_dotenv
load_dotenv() 
from langchain_groq import ChatGroq

api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="Gemma-7b-It")


## Stuff Document Chain Text summarization

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ML_Overview.pdf")
docs = loader.load_and_split()


In [34]:
from langchain.prompts import PromptTemplate

template = """ Write a concise and short summary of the following ,
Doc: {text}
"""
prompt = PromptTemplate(input_variables=['text'], template=template)


In [35]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type='stuff', prompt=prompt, verbose=True)
output_summary = chain.run(docs) 




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and short summary of the following ,
Doc: An Extensive Overview of Machine Learning
Machine Learning (ML) is a field of artificial intelligence (AI) that gives computers the ability to learn
from data without being explicitly programmed. It's one of the fastest-growing fields in technology,
driving innovations in various industries like healthcare, finance, retail, and autonomous vehicles.
In this extensive overview, we will delve into the following topics:
1. Introduction to Machine Learning
2. Types of Machine Learning
3. Key Algorithms in Machine Learning
4. Applications of Machine Learning
5. Challenges and Limitations
6. Deep Learning and Neural Networks
7. Future Trends in Machine Learning
---
1. Introduction to Machine Learning
Machine Learning is the science of getting computers to act without being explicitly programmed. In
the past decade, machine learnin

### Map-Reduce chain For Summarization

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

final_documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)


In [38]:
chunks_prompt="""
Please summarize the below document:
Doc:`{text}'
Summary:
"""

final_prompt='''
Provide the final summary of the entire document with these important points.
Add a Title,Start the precise summary with an introduction and provide the summary in number 
points.
Doc:{text}

'''

map_prompt_template = PromptTemplate(input_variables=['text'], template=chunks_prompt)
final_prompt_template = PromptTemplate(input_variables=['text'], template=final_prompt)

summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output = summary_chain.run(final_documents)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below document:
Doc:`An Extensive Overview of Machine Learning
Machine Learning (ML) is a field of artificial intelligence (AI) that gives computers the ability to learn
from data without being explicitly programmed. It's one of the fastest-growing fields in technology,
driving innovations in various industries like healthcare, finance, retail, and autonomous vehicles.
In this extensive overview, we will delve into the following topics:
1. Introduction to Machine Learning
2. Types of Machine Learning
3. Key Algorithms in Machine Learning
4. Applications of Machine Learning
5. Challenges and Limitations
6. Deep Learning and Neural Networks
7. Future Trends in Machine Learning
---
1. Introduction to Machine Learning
Machine Learning is the science of getting computers to act without being explicitly programmed. In
the past decade, machine learning has given 

### Refine Chain For Summarization

In [39]:
chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True)
output_summary = chain.run(final_documents)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"An Extensive Overview of Machine Learning
Machine Learning (ML) is a field of artificial intelligence (AI) that gives computers the ability to learn
from data without being explicitly programmed. It's one of the fastest-growing fields in technology,
driving innovations in various industries like healthcare, finance, retail, and autonomous vehicles.
In this extensive overview, we will delve into the following topics:
1. Introduction to Machine Learning
2. Types of Machine Learning
3. Key Algorithms in Machine Learning
4. Applications of Machine Learning
5. Challenges and Limitations
6. Deep Learning and Neural Networks
7. Future Trends in Machine Learning
---
1. Introduction to Machine Learning
Machine Learning is the science of getting computers to act without being explicitly programmed. In
the past decade, machine learning has given u